<a href="https://colab.research.google.com/github/ipediz/BachehayekhafaneTMU/blob/main/_above_60_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code was written with the aid of ChatGPT**

In [ ]:
#All states population %of above 60
!pip install Census

In [ ]:
from census import Census
import pandas as pd

# Replace 'YOUR_CENSUS_API_KEY' with your actual API key
c = Census("b6530e19ac88a7f5b6c865d194b4f0c9ae06e1ae")

# List of FIPS codes for all states
state_fips = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17',
              '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',
              '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46',
              '47', '48', '49', '50', '51', '53', '54', '55', '56']

# Function to get population above 60 for a given state and year from ACS 5-year data
def get_population_over_60(state, year):
    # List of variables for males and females above 60 (ACS variables)
    variables = [
        'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E',  # Males 60+
        'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E',  # Females 60+
        'B01001_001E'  # Total population
    ]

    # Fetching data from the ACS 5-year estimate dataset for the given year
    try:
        data = c.acs5.state_county(variables, state, Census.ALL)
    except Exception as e:
        print(f"Error fetching data for state {state}, year {year}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

    df = pd.DataFrame(data)

    # Calculate percentage of population over 60
    males_over_60 = (df['B01001_020E'].astype(float) + df['B01001_021E'].astype(float) +
                     df['B01001_022E'].astype(float) + df['B01001_023E'].astype(float) +
                     df['B01001_024E'].astype(float) + df['B01001_025E'].astype(float))

    females_over_60 = (df['B01001_044E'].astype(float) + df['B01001_045E'].astype(float) +
                       df['B01001_046E'].astype(float) + df['B01001_047E'].astype(float) +
                       df['B01001_048E'].astype(float) + df['B01001_049E'].astype(float))

    total_population_over_60 = males_over_60 + females_over_60
    df['percent_over_60'] = (total_population_over_60 / df['B01001_001E'].astype(float)) * 100

    # Add state and year information to the DataFrame
    df['state'] = state
    df['year'] = year

    return df[['state', 'county', 'percent_over_60', 'year']]

# Loop through each year and state to collect the data
def collect_data(years):
    all_data = []
    for year in years:
        for state in state_fips:
            try:
                df = get_population_over_60(state, year)
                if not df.empty:
                    all_data.append(df)
            except Exception as e:
                print(f"Error fetching data for state {state} in {year}: {e}")
    return pd.concat(all_data)

# Get data for the ACS 5-year estimates corresponding to 2008 to 2015
years = range(2008, 2016)
all_data = collect_data(years)

# Reset index and print the data
all_data.reset_index(drop=True, inplace=True)
print(all_data)

# Save the data to a CSV file
all_data.to_csv('population_over_60_2008_2015.csv', index=False)


In [ ]:
# Assuming 'all_data' is the output from your original code

# Create an empty list to store the expanded data
expanded_data = []

# Iterate over each unique year and county
for _, row in all_data.iterrows():
    # For each row, create 4 quarters
    for quarter in range(1, 5):
        new_row = row.copy()
        new_row['Quarter'] = quarter
        expanded_data.append(new_row)

# Convert the list of expanded data into a DataFrame
all_data_expanded = pd.DataFrame(expanded_data)

# Print the resulting DataFrame
print(all_data_expanded)


In [ ]:
# Assuming 'all_data_expanded' is the output from your expanded data

# Combine the 'state' and 'county' columns to create the FIPS code
# Ensure both state and county codes are strings, and pad county codes to 3 digits
all_data_expanded['FIPS'] = all_data_expanded['state'].str.zfill(2) + all_data_expanded['county'].str.zfill(3)

# Drop the old 'state' and 'county' columns if you only need the FIPS code
all_data_expanded = all_data_expanded.drop(columns=['state', 'county'])

# Reorder columns if needed (FIPS, year, Quarter, percent_over_60)
all_data_expanded = all_data_expanded[['FIPS', 'year', 'Quarter', 'percent_over_60']]

# Display the resulting DataFrame
print(all_data_expanded)
